In [1]:
import torch
import torch.nn.functional as F
from torch import nn

## 1 不含模型参数的自定义层

In [5]:
class CenteredLayer(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, X):
        return X - X.mean()

In [8]:
layer = CenteredLayer()
layer(torch.FloatTensor([1, 2, 3, 4, 5]))

tensor([-2., -1.,  0.,  1.,  2.])

In [9]:
net = nn.Sequential(nn.Linear(8, 128), CenteredLayer())
Y = net(torch.rand(4, 8))
Y.mean()

tensor(0., grad_fn=<MeanBackward0>)

## 2 含模型参数的自定义层

In [13]:
class MyLinear(nn.Module):
    def __init__(self, in_units, units):
        super().__init__()
        self.weight = nn.Parameter(torch.randn(in_units, units)) # 放进Parameter自动设置求导
        self.bias = nn.Parameter(torch.randn(units,))
    def forward(self, X):
        # 不知道为啥这里用的是data，这样不就取消自动求导，但实际上还是自动求导的
        linear = torch.matmul(X, self.weight.data) + self.bias.data
        return F.relu(linear)

In [14]:
linear = MyLinear(5, 3)
linear.weight

Parameter containing:
tensor([[ 0.4355, -0.3428, -0.9911],
        [ 0.5708,  0.2869, -0.4492],
        [-0.8862,  2.3849, -0.6174],
        [-0.7159,  0.9893, -1.1692],
        [-1.3504, -2.3015,  0.9879]], requires_grad=True)